In [3]:
import pandas as pd
import numpy as np
from math import sqrt

np.set_printoptions(precision=2)  # 소수점 2째짜리까지만 출력할 꺼시다

In [27]:
RATING_DATA_PATH = './data/ratings.csv'
rating_data = pd.read_csv(RATING_DATA_PATH, index_col='user_id')
rating_data.head()



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
user_id,,,,,,,,,,,,,,,,,,,,
0,2.0,3.0,4.0,NaN,2.0,3.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,1.0,NaN,NaN,2.0,5.0,2.0,NaN
1,NaN,NaN,NaN,4.0,NaN,5.0,NaN,NaN,2.0,NaN,4.0,NaN,1.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN
2,2.0,NaN,1.0,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,5.0,3.0,3.0,NaN,3.0,3.0,4.0,NaN,NaN
3,NaN,3.0,5.0,NaN,NaN,NaN,4.0,3.0,NaN,5.0,3.0,NaN,4.0,NaN,NaN,2.0,NaN,NaN,NaN,2.0
4,NaN,NaN,NaN,5.0,NaN,NaN,2.0,2.0,2.0,NaN,3.0,4.0,2.0,5.0,4.0,NaN,NaN,NaN,4.0,2.0


In [9]:
def distance(user_1, user_2):
    """유클리드 거리를 계산해주는 해 줄 꺼시다~ """
    return sqrt(np.sum((user_1 - user_2)**2))

In [28]:
def filter_users_without_movie(rating_data, movie_id):
    """movie_id 번째 영화를 평가하지 않은 유저들은 미리 제외할 꺼시다"""
    
    return rating_data[~np.isnan(rating_data[:,movie_id])] # 여기서 NaN = False



In [11]:
def fill_nan_with_user_mean(rating_data):
    """평점 데이터의 빈값들을 각 유저 평균 값으로 채워줄 꺼시다 """
    filled_data = np.copy(rating_data)  # 평점 데이터를 훼손하지 않기 위해 복사
    row_mean = np.nanmean(filled_data, axis=1)  # 유저 평균 평점 계산
    
    inds = np.where(np.isnan(filled_data))  # 비어 있는 인덱스들을 구한다
    filled_data[inds] = np.take(row_mean, inds[0])  #빈 인덱스를 유저 평점으로 채운다
    
    return filled_data

In [12]:
def get_k_neighbors(user_id, rating_data, k):
    """user_id에 해당하는 유저의 이웃들(K개)을 찾아줄 꺼시다"""
    distance_data = np.copy(rating_data)  # 평점 데이터를 훼손하지 않기 위해 복사
    # 마지막에 거리 데이터를 담을 열 추가한다
    distance_data = np.append(distance_data, np.zeros((distance_data.shape[0], 1)), axis=1)
   
    for i in range(len(distance_data)):
        row = distance_data[i]
        
        if i == user_id:  # 같은 유저면 거리를 무한대로 설정
            row[-1] = np.inf
        else:  # 다른 유저면 마지막 열에 거리 데이터를 저장
            row[-1] = distance(distance_data[user_id][:-1], row[:-1])
    
    # 데이터를 거리 열을 기준으로 정렬한다
    distance_data = distance_data[np.argsort(distance_data[:, -1])]
    
    # 가장 가까운 k개의 행만 리턴한다 + 마지막(거리) 열은 제외한다
    return distance_data[:k, :-1]

In [14]:
def predict_user_rating(rating_data, k, user_id, movie_id,):
    """예측 행렬에 따라 유저의 영화 평점 예측 값 구하기"""
    # movie_id 번째 영화를 보지 않은 유저를 데이터에서 미리 제외시킨다
    filtered_data = filter_users_without_movie(rating_data, movie_id)
    # 빈값들이 채워진 새로운 행렬을 만든다
    filled_data = fill_nan_with_user_mean(filtered_data)
    # 유저 user_id와 비슷한 k개의 유저 데이터를 찾는다
    neighbors = get_k_neighbors(user_id, filled_data, k)
    
    return np.mean(neighbors[:,movie_id])

In [15]:
rating_data = pd.read_csv(RATING_DATA_PATH, index_col='user_id').values
predict_user_rating(rating_data, 10, 0, 2)  

2.8